# Yale Faces 

## Importing Libraries

In [4]:
%matplotlib inline
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import scipy.misc
import os
import warnings
warnings.filterwarnings("ignore")


## Reading the Dataset: Yale Faces

In [5]:
#Create the FaceCascader
cascPath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)
#Initialize the number of persons
persons = 0
#Loading the data
full_face_yale = []
full_labels_yale = []
all_images_yale = os.listdir('yalefaces')
for j in range(1,16):    # there are 15 different people in the dataset
    img_list = [filename for filename in all_images_yale if filename.startswith('subject%.2d'%j)]
    face1 = np.zeros((len(img_list),80,60))
    for k,img in enumerate(img_list):
        I = scipy.misc.imread(os.path.join('yalefaces',img))
        faces = faceCascade.detectMultiScale(
            I,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        x,y,h,w = faces[0]
        face1[k,...] = zoom(I[y:y+h,x:x+w],[80./h,60./w])
    labels = (j-1)*np.ones((11,))
    full_face_yale.append(face1)
    full_labels_yale.append(labels)
full_face_yale = np.stack(full_face_yale)    # contains the images of all faces
full_labels_yale = np.stack(full_labels_yale)   # contains the ID of all the faces
persons = len(full_labels_yale)

In [6]:
persons

15

In [7]:
#Get the shape
print(full_face_yale.shape)
print(full_labels_yale.shape)

(15, 11, 80, 60)
(15, 11)


# Reading the Dataset: JaffedBase

In [8]:
#Get all the images
all_images_jaffe = os.listdir('jaffedbase/jaffe')
img_list = [filename for filename in all_images_jaffe if filename.endswith('.tiff')]
#Get the names and poses
names = [] 
poses = []
for i in range(len(img_list)):
    names.append(str.split(img_list[i], sep = '.')[0])
    poses.append(str.split(img_list[i],sep = '.')[1][0:2])
#Store the unique names and poses
names = np.unique(np.array(names))
poses = np.unique(np.array(poses))

In [9]:
#Store the images of every Japanese model along with their respective poses
full_face_jaffed = np.zeros(shape = (len(names),len(poses),80,60))
full_labels_jaffed = np.zeros(shape=(len(names),len(poses)))
for i,name in enumerate(names):
    images_person = [filename for filename in all_images_jaffe if filename.startswith(name)]
    for j,pose in enumerate(poses):
        images_person_pose = [filename for filename in images_person if filename.startswith(name+'.'+ pose)]
        particular_image = images_person_pose[0]
        I = scipy.misc.imread(os.path.join(os.getcwd()+'\jaffedbase\jaffe',particular_image))
        if(len(I.shape) == 3):
            I = I[...,0]
        faces = faceCascade.detectMultiScale(
            I,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        x,y,h,w = faces[0]
        face1= zoom(I[y:y+h,x:x+w],[80./h,60./w])
        full_face_jaffed[i,j,...] = face1
        full_labels_jaffed[i,j] = i+persons #Jaffed Database represents 15th person onwards

In [10]:
persons = persons + len(full_labels_jaffed)
print(persons)

15


# Reading the Dataset: Faces94

In [11]:
#Get all the images: 10 female, 10 male

In [12]:
all_images_94_female = os.listdir('faces94/faces94/female')
all_images_94_male = os.listdir('faces94/faces94/male')
female_img_list = [filename for filename in all_images_94_female]
male_img_list = [filename for filename in all_images_94_male]
#Take the first 3 image files
female_img_list = female_img_list[0:10]
male_img_list = male_img_list[0:10]

In [13]:
#Number of individuals: 10 - 5 female, 5 male
#Number of poses: 10
#Read in female data
full_faces_94_female = np.zeros(shape = (10,10,80,60))
full_labels_94_female = np.zeros(shape = (10,10))
full_faces_94_male = np.zeros(shape = (10,10,80,60))
full_labels_94_male = np.zeros(shape = (10,10))
path = os.getcwd() +'/faces94/faces94/female'
for i,indiv_image in enumerate(female_img_list):
    path1 = path + '/'+ indiv_image
    for j in range(7):
        img = os.listdir(path1)[j]    
        I = scipy.misc.imread(os.path.join(path1,img),flatten = True)
        I = cv2.resize(I, (60,80))
        full_faces_94_female[i,j,...] = I
        full_labels_94_female[i,j] = i+persons 
persons = persons + len(full_labels_94_female)
#Read in male data
for i,indiv_image in enumerate(female_img_list):
    path1 = path + '/'+ indiv_image
    for j in range(7):
        img = os.listdir(path1)[j]    
        I = scipy.misc.imread(os.path.join(path1,img),flatten = True)
        I = cv2.resize(I, (60,80))
        full_faces_94_male[i,j,...] = I
        full_labels_94_male[i,j] = i+persons
full_faces_94 = np.vstack((full_faces_94_female,full_faces_94_male))
full_labels_94 = np.vstack((full_labels_94_female,full_labels_94_male))

In [14]:
full_faces_94.shape

(20, 10, 80, 60)

# Consolidate the Dataset

In [15]:
#Copy
full_face = full_face_yale
full_labels = full_labels_yale
del full_face_yale
del full_labels_yale

In [16]:
train_labels

NameError: name 'train_labels' is not defined

In [17]:
%matplotlib inline
cv2.imshow("aa",train_set[1])
cv2.waitKey(10)
cv2.destroyAllWindows()

NameError: name 'train_set' is not defined

In [18]:
#Function for Gaussian Blurring
def gaussian_blur(full_face):
    blur = np.zeros(full_face.shape)
    for i in range(full_face.shape[0]):
        blur[i,...]=cv2.GaussianBlur(full_face[i,...],(5,5),0)
    return(blur)

In [19]:
# divide the data into train and test
# pick 2 images from each dataset and put it into test set

#Training Data
train_set = np.zeros((full_face.shape[0],9,80,60))
train_set_jaffed = np.zeros((full_face_jaffed.shape[0],5,80,60))
train_set_94 = np.zeros((full_faces_94.shape[0],8,80,60))
#Test Data
test_set = np.zeros((full_face.shape[0],2,80,60))
test_set_jaffed =  np.zeros((full_face_jaffed.shape[0],2,80,60))
test_set_94 = np.zeros((full_faces_94.shape[0],2,80,60))
#Train Labels
train_labels = np.zeros((full_face.shape[0],9))
train_labels_jaffed = np.zeros((full_face_jaffed.shape[0],5))
train_labels_94 = np.zeros((full_faces_94.shape[0],8))
#Test Labels
test_labels = np.zeros((full_face.shape[0],2))
test_labels_jaffed = np.zeros((full_face_jaffed.shape[0],2))
test_labels_94 = np.zeros((full_faces_94.shape[0],2))
#Loop for Yale Faces A
for k in range(full_face.shape[0]):
    g = np.random.permutation(11)   # randomly select the 9 faces
    train_set[k,...] = full_face[k,g[:9],...]
    train_labels[k,...] = full_labels[k,g[:9]]
    test_set = full_face[:,g[9:],...]
    test_labels[k,...] = full_labels[k,g[9:]]
    
#Loop for Japanese 
for k in range(full_face_jaffed.shape[0]):
    g = np.random.permutation(7)   # randomly select the 5 faces
    train_set_jaffed[k,...] = full_face_jaffed[k,g[:5],...]
    train_labels_jaffed[k,...] = full_labels_jaffed[k,g[:5]]
    test_set_jaffed = full_face_jaffed[:,g[5:],...]
    test_labels_jaffed[k,...] = full_labels_jaffed[k,g[5:]]
    
#Loop for Faces_94
for k in range(full_faces_94.shape[0]):
    g = np.random.permutation(10)   # randomly select the 8 faces
    train_set_94[k,...] = full_faces_94[k,g[:8],...]
    train_labels_94[k,...] = full_labels_94[k,g[:8]]
    test_set_94 = full_faces_94[:,g[8:],...]
    test_labels_94[k,...] = full_labels_94[k,g[8:]]

In [20]:
#Apply Gaussian Blur
#train_set = gaussian_blur(train_set)
#test_set = gaussian_blur(test_set)

In [21]:
train_set.shape

(15, 9, 80, 60)

# Feature Extraction

In [22]:
from skimage.feature import local_binary_pattern
#Reshaping Train Set
train_set = np.reshape(train_set,[-1,80,60])
train_set_jaffed = np.reshape(train_set_jaffed,[-1,80,60])
train_set_94 = np.reshape(train_set_94,[-1,80,60])
train_set = np.vstack((train_set,train_set_jaffed,train_set_94))
del train_set_jaffed
#Reshaping Train Labels
train_labels = np.reshape(train_labels,[-1,])
train_labels_jaffed = np.reshape(train_labels_jaffed,[-1,])
train_labels_94 = np.reshape(train_labels_94,[-1,])
train_labels = np.hstack((train_labels,train_labels_jaffed,train_labels_94))
del train_labels_jaffed
#Reshaping Test Set
test_set = np.reshape(test_set,[-1,80,60])
test_set_jaffed = np.reshape(test_set_jaffed,[-1,80,60])
test_set_94 = np.reshape(test_set_94,[-1,80,60])
test_set = np.vstack((test_set,test_set_jaffed))
del test_set_jaffed
#Reshaping Test Labels
test_labels = np.reshape(test_labels,[-1,])
test_labels_jaffed = np.reshape(test_labels_jaffed,[-1,])
test_labels_94 = np.reshape(test_labels_94,[-1,])
test_labels = np.hstack((test_labels,test_labels_jaffed,test_labels_94))
del test_labels_jaffed
train_feat = np.zeros_like(train_set)
test_feat = np.zeros_like(test_set)

for k in range(train_feat.shape[0]):
        train_feat[k,...] = local_binary_pattern(train_set[k,...],8,3)#,method='uniform')
for k in range(test_feat.shape[0]):
        test_feat[k,...] = local_binary_pattern(test_set[k,...],8,3)#,method='uniform')
print(np.amax(test_feat))

255.0


# Hog Features

In [23]:
from skimage.feature import hog,corner_fast
#features_nums = faces.data.reshape(-1,62,47)
train_hog_features = [hog(train_set[n,:,:],pixels_per_cell=(7,7)) for n in range(train_feat.shape[0])]
train_hog_features = np.stack(train_hog_features,0)

train_hog_features.shape

test_hog_features = [hog(test_set[n,:,:],pixels_per_cell=(7,7)) for n in range(test_feat.shape[0])]
test_hog_features = np.stack(test_hog_features,0)

test_hog_features.shape

(30, 4374)

# LBP

In [24]:
# set the number of grids in x and y direction
# divide the image into the specified number of grids
# for each grid compute the histogram with 64 bins
# concatenate the histogram from all grid regions to form a single feature
# now we have one feature vector per image
# Note that grid size and histogram bin sizes are variable
x_grid = 7
y_grid = 7
grid_size_x = train_feat.shape[1]//x_grid
grid_size_y = train_feat.shape[2]//y_grid
for k in range(train_feat.shape[0]):
    for x in range(x_grid):
        for y in range(y_grid):
            patch = train_feat[k,x*grid_size_x:(x+1)*grid_size_x,y*grid_size_y:(y+1)*grid_size_y]
            hist,_ = np.histogram(patch.reshape(-1),bins=64)
            if x==0 and y==0:
                patch_hist_list = hist
            else:
                patch_hist_list = np.concatenate([patch_hist_list,hist],0)
    if k==0:
        data_hist = patch_hist_list[np.newaxis,...]
    else:
        data_hist = np.concatenate([data_hist,patch_hist_list[np.newaxis,...]],0)
print(data_hist.shape)

grid_size_x = test_feat.shape[1]//x_grid
grid_size_y = test_feat.shape[2]//y_grid

for k in range(test_feat.shape[0]):
    for x in range(x_grid):
        for y in range(y_grid):
            patch = test_feat[k,x*grid_size_x:(x+1)*grid_size_x,y*grid_size_y:(y+1)*grid_size_y]
            hist,_ = np.histogram(patch.reshape(-1),bins=64)
            if x==0 and y==0:
                patch_hist_list = hist
            else:
                patch_hist_list = np.concatenate([patch_hist_list,hist],0)
    if k==0:
        test_data_hist = patch_hist_list[np.newaxis,...]
    else:
        test_data_hist = np.concatenate([test_data_hist,patch_hist_list[np.newaxis,...]],0)
print(test_data_hist.shape)

(295, 3136)
(30, 3136)


# PCA (Hog to LBP)

In [25]:
from sklearn.decomposition import PCA
pca = PCA(whiten=True)
#test_feat = test_hog_features
#train_feat = train_hog_features
train_feat = data_hist
test_feat = test_data_hist
print(train_feat.shape,test_feat.shape)
pca.fit(train_feat)

((295, 3136), (30, 3136))


PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=True)

In [26]:
# select number of components which explain 99% of the data variance
n_components = np.argmin(np.cumsum(pca.explained_variance_ratio_)<=0.99)
print(n_components)
pca = PCA(n_components)

train_feat_pca = pca.fit_transform(train_feat)
#train_feat_pca = train_feat_pca[:,:n_components]
test_feat_pca = pca.transform(test_feat)
#test_feat_pca = test_feat_pca[:,:n_components]
print(train_feat_pca.shape)

156
(295, 156)


# Make pairs of images with labels as positive pairs and negative pairs

In [27]:
import itertools
img_pair = []
img_full_pair = []
label_pair = []
for pair in itertools.combinations_with_replacement(range(train_feat_pca.shape[0]),2):
    img_pair.append(train_feat_pca[pair,:])
    #img_full_pair.append(train_set[pair,:])
    label_pair.append(train_labels[pair,])
img_pair = np.stack(img_pair)


In [28]:
#Careful of Memory Storage
#img_full_pair = np.stack(img_full_pair)
label_pair = np.stack(label_pair)
label_pair = (label_pair[:,0] == label_pair[:,1])
label_pair = label_pair*1

In [29]:
test_img_pair = []
test_label_pair = []
for pair in itertools.combinations_with_replacement(range(test_feat.shape[0]),2):
    test_img_pair.append(test_feat_pca[pair,...])
    test_label_pair.append(test_labels[pair,])
test_img_pair = np.stack(test_img_pair)
test_label_pair = np.stack(test_label_pair)
test_label_pair = (test_label_pair[:,0] == test_label_pair[:,1])
test_label_pair = test_label_pair*1

In [30]:
img_pair = np.abs(img_pair[:,0,...] - img_pair[:,1,...])
test_img_pair = np.abs(test_img_pair[:,0,...] - test_img_pair[:,1,...])

# Shuffle the Training Data and train the classifier

In [31]:
# shuffle the training data
g = np.random.permutation(img_pair.shape[0])
img_pair = img_pair[g,:]
label_pair = label_pair[g,]

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 20)
rf.fit(img_pair,label_pair)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
label_pair.shape

(43660,)

# Hyper-Parameter Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_estimators' : [10,20,30,40,50,100],'criterion' : ['entropy','gini']}
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid)
grid_search.fit(img_pair,label_pair)
print(grid_search.best_params_)
print(grid_search.best_score_)

# Evaluation 

In [115]:
print(rf.score(test_img_pair,test_label_pair))

0.9870967741935484


In [116]:
pred = rf.predict_proba(test_img_pair)
from sklearn.metrics import f1_score
print(f1_score(test_label_pair,rf.predict(test_img_pair)))

0.9285714285714286


In [117]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(test_label_pair,rf.predict(test_img_pair)))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99       420
          1       1.00      0.87      0.93        45

avg / total       0.99      0.99      0.99       465



In [118]:
print(confusion_matrix(test_label_pair,rf.predict(test_img_pair)))

[[420   0]
 [  6  39]]


In [119]:
from sklearn.metrics import auc,precision_recall_curve
precision,recall,thr = precision_recall_curve(test_label_pair, pred[:,1], pos_label=1)
print('Number of thresholds evaluated: ',len(thr))
plt.plot(recall,precision)
plt.xlabel('recall')
plt.ylabel('precision')
plt.show()

('Number of thresholds evaluated: ', 37)


RuntimeError: Could not create write struct

In [121]:
f1_score = (2*precision*recall)/(precision+recall)
print('max f1-score:',np.amax(f1_score))
pr_auc = auc(recall,precision)
print('Area under Precision Recall Curve: ',pr_auc)

('max f1-score:', 0.9411764705882353)
('Area under Precision Recall Curve: ', 0.9502236292103102)


# Testing

In [39]:
import h5py

filename = "test1.h5"
f = h5py.File(filename, 'r')

In [40]:
data = []
label = []
keys = list(f.keys())

for i in range(len(keys)):
    a_group_key = keys[i]
    data.append(f[a_group_key][()])
    label.append(a_group_key)
    
import pandas as pd
pairs = pd.read_csv("image_pairs.csv")
pair1 = []
pair2 = []

for i in range(len(pairs)):
    if pairs["pair1"][i] > 100:
        pair1.append(pairs["pair1"][i] - 100)
    else:
        pair1.append(pairs["pair1"][i] - 20)
        
    if pairs["pair2"][i] > 100:
        pair2.append(pairs["pair2"][i] - 100)
    else:
        pair2.append(pairs["pair2"][i] - 20)
        
        
pair3 = np.column_stack((pair1, pair2))
img_full_pair1 = []
img_full_pair2 = []
for x in pair3:
    img_full_pair1.append(data[x[0]])
    img_full_pair2.append(data[x[1]])
    

In [41]:
face1 = np.zeros((len(data),80,60))
full_face = []

for i in range(len(data)):
        #face1[i,...] = cv2.resize(data[15], (60, 80))
        faces = faceCascade.detectMultiScale(
            data[i],
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30))
        if len(faces) == 0:
            face1[i,...] = cv2.resize(data[i], (60, 80))
        else:

            x,y,h,w = faces[0]
            face1[i,...] = zoom(data[i][y:y+h,x:x+w],[80./h,60./w])
    #labels = (j-1)*np.ones((11,))
full_face.append(face1)
        #full_labels.append(labels)
full_face = np.stack(full_face)    # contains the images of all faces
#full_labels = np.stack(full_labels)   # contains the ID of all the faces

# LBP and Hog

In [42]:
#Gaussian Blur
#face1 = gaussian_blur(face1)

from skimage.feature import hog,corner_fast
patch_shape = []


x_grid = 7
y_grid = 7
grid_size_x = face1.shape[1]//x_grid
grid_size_y = face1.shape[2]//y_grid
for k in range(face1.shape[0]):
    for x in range(x_grid):
        for y in range(y_grid):
            patch = face1[k,x*grid_size_x:(x+1)*grid_size_x,y*grid_size_y:(y+1)*grid_size_y]
            hist,_ = np.histogram(patch.reshape(-1),bins=64)
            patch_shape.append(np.shape(patch))
           
           
            #hog(features_nums[n,:,:],pixels_per_cell=(7,7)
            if x==0 and y==0:
                patch_hist_list = hist
            else:
                patch_hist_list = np.concatenate([patch_hist_list,hist],0)
    if k==0:
        data_hist = patch_hist_list[np.newaxis,...]
    else:
        data_hist = np.concatenate([data_hist,patch_hist_list[np.newaxis,...]],0)
print(data_hist.shape)


from skimage.feature import hog,corner_fast
#features_nums = faces.data.reshape(-1,62,47)
hog_features = [hog(face1[n,:,:],pixels_per_cell=(7,7)) for n in range(face1.shape[0])]
hog_features = np.stack(hog_features,0)


(120, 3136)


In [43]:
#train_feat1 = hog_features
train_feat1 = data_hist

train_feat_pca1 = pca.transform(train_feat1)
print(train_feat_pca1.shape)

(120, 156)


In [44]:
test_img = []
for i in range(len(pair1)):
    test_img.append(np.abs(train_feat_pca[pair1[i]] - train_feat_pca[pair2[i]]))

In [45]:
pred = rf.predict(test_img)
sum(pred)

418

In [49]:
import pandas as pd 
df = pd.DataFrame(pred)
df.index.name = 'Id'
df.columns = ['Expected']
df.to_csv("sub5.csv")

In [51]:
for i in range(len(df)):
    df['Expected'][i] = 0

In [54]:
df.to_csv("sub5.csv")

In [56]:
0.09*len(pred)

318.59999999999997

In [59]:
train_feat_pca.shape

(295, 156)